# Test the pipeline with the Weighted Kalman Filter (WKF).

In [1]:
import os
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
import sys
import matplotlib.pyplot as plt

# Get the path of the project root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# Add the project root to sys.path if it is not already in sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the ImageSequenceDataset and create_dataloader from the pipeline module
from seismogram_extraction.pipeline import evaluate_filter, create_dataloader
from seismogram_extraction.filters.weighted_kalman_filter import WeightedKalmanFilter

import torch.multiprocessing as mp

mp.set_start_method('spawn')  # or 'forkserver' if available

In [2]:
### Parameters
Dt = 0.5 # np.linspace(0.5, 5, 2) # Time step related to the state transition matrix A, ! different than sampling rate dt of signal s

# Assuming no process noise
sigma_p = 0.01 # np.linspace(1e-2, 2, 2) 
sigma_v = 2 # np.linspace(1e-2, 2, 2)

# Assuming no measurement noise
sigma_z = 0.25 # np.linspace(1e-6, 1, 5)

p_fa = 0.0001 # np.linspace(1e-4, 1, 5)
###

A = np.array([[1, Dt],
              [0, 1]]).astype(np.float64)
                        
H = np.array([[1, 0]]).astype(np.float64)

Q = np.array([[sigma_p**2, 0],
              [0, sigma_v**2]])

R = np.array([[sigma_z**2]])

# Initial state covariance given all_x_0
P = np.zeros((2, 2))
P[1, 1] = 10

image_folder_path = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-00\signals"
GTs_folder_path = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-00\ground_truth"
    
dataloader = create_dataloader(image_folder_path, GTs_folder_path, batch_size=4)

output_folder_path = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines\overlap_0-00\processed_WKF"
os.makedirs(output_folder_path, exist_ok=True)
    
evaluate_filter(image_folder_path, GTs_folder_path, output_folder_path, WeightedKalmanFilter(A, H, Q, R, p_fa))


(nan, 0.0)